In [1]:
import os
import re
import time
import requests
import pandas as pd
import ast
import json

In [21]:
class DictionaryManagement:

    def __init__(self):
        self.Dictionary = {}
    
    def FindAbbreviations(self, text):
        # (?=[A-Z0-9]*[A-Z])
        # İfadenin devamında sıfır veya daha fazla "büyük harf ve rakam" ve ardından en az bir büyük harf olmalıdır.
        #
        # [A-Z0-9]{2,}
        # En az bir büyük harf ve rakamdan oluşan, en az 2 karakterlik veri
        #
        # Kombinasyonlu ifade de bu ifadelerin arasına "/" karakteri gelirse de kabul ediyor.
        # DM2, PCRE, DM2/A (kombinasyonlu)
        abbreviation_pattern = r'\b(?=[A-Z0-9]*[A-Z])[A-Z0-9]{2,6}\b'
        combined_pattern = r'\b(?=[A-Z0-9]*[A-Z])[A-Z0-9]{2,6}/(?=[A-Z0-9]*[A-Z])[A-Z0-9]{1,5}\b'
    
        tokens = re.findall(abbreviation_pattern, text)
        tokens.extend(re.findall(combined_pattern, text))
    
        abbreviations = {token for token in tokens if re.search(r'[A-Z]', token) and not token[0].isdigit()}
    
        return sorted(abbreviations)

    def FilterAbbreviationsWithTreshold(self, allAbbreviations, threshold):
        # Çok fazla kısaltma bulunurken, bunların bazıları yanlış olabilir.
        # Az sayıda tekrar eden veri varsa bunları sileceğiz, belirli bir eşik değerin üstündeki
        # tekrar sayıları o verinin potansiyel kısaltma olduğunu ifade edeceğini kabul ediyoruz.
        abbreviationsCount = {}
    
        for abbreviation in allAbbreviations:
            abbreviationsCount[abbreviation] = abbreviationsCount.get(abbreviation, 0) + 1
    
        return [abbr for abbr, count in abbreviationsCount.items() if count > threshold]

    def SetDictionary(self, di):
        self.Dictionary = di

    def GetAllAbbreviations(self, text):
        arr = list(set(self.FindAbbreviations(text)))
        result = {}
        for key in arr:
            if key in self.Dictionary:
                result[key] = self.Dictionary[key][:3]
        return result

In [22]:
DManager = DictionaryManagement()

with open("Dictionary.json", "r", encoding="utf-8") as json_file:
    DManager.SetDictionary(json.load(json_file))

In [26]:
DManager.GetAllAbbreviations("İlaç her gün sabah PO ve AV ile CHF")

{'PO': [{'en_text': 'Per Oral', 'tr_text': 'Ağızdan', 'source': 'i2b2'},
  {'en_text': 'Per Oral', 'tr_text': 'Oral', 'source': 'i2b2'}],
 'AV': [{'en_text': 'arteriovenous',
   'tr_text': 'arteriyovenöz',
   'source': 'i2b2'}],
 'CHF': [{'en_text': 'congestive heart failure',
   'tr_text': 'Konjestif kalp yetmezliğiKKY',
   'source': 'i2b2'}]}